In [ ]:
# Nov 27 2020
# Big Data Project 
# A small neural network

References:

https://keras.io/about/

https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

In [1]:
import csv 
import numpy as np
import random

import keras
from keras.models import Sequential
from keras.layers import Dense

In [2]:
filename = "cps3_reshape.csv"

data = [] 
  
# opening the CSV file 
with open(filename, mode ='r') as file:     
         
       # reading the CSV file 
       csvFile = csv.reader(file) 
  
       # displaying the contents of the CSV file 
       for line in csvFile: 
            data.append(line)

In [3]:
data[0][45]=('BIRTH')
for i in range(len(data[1:])):
    person = data[i+1]
    if float(person[41])>float(person[21]):
            person.append('1') #birth
            #print(len(person))
    elif float(person[41])==float(person[21]):
            person.append('0') #no birth
    else:
            person.append('9') #error

In [4]:
# remove minors from data
# originally there were 47000 rows 
# after this there are 36770
def remove_minors(data):
    data2 = []
    data2.append(data[0])
    for row in data[1:]:
        if int(row[15])>17:
            data2.append(row)
    return data2

data = remove_minors(data)

In [6]:
print(data[0])
print(len(data))

['YEAR', 'SERIAL', 'MONTH', 'HWTFINL', 'CPSID', 'REGION', 'STATEFIP', 'METRO', 'FAMINC', 'HRHHID', 'HRHHID2', 'PERNUM', 'WTFINL', 'CPSIDP', 'RELATE', 'AGE', 'SEX', 'RACE', 'MARST', 'POPSTAT', 'VETSTAT', 'NCHILD', 'BPL', 'YRIMMIG', 'CITIZEN', 'NATIVITY', 'HISPAN', 'EMPSTAT', 'LABFORCE', 'OCC', 'IND', 'CLASSWKR', 'UHRSWORKT', 'WHYUNEMP', 'WKSTAT', 'PROFCERT', 'EDUC', 'SCHLCOLL', 'DIFFANY', 'FAMINC2', 'MARST2', 'NCHILD2', 'EMPSTAT2', 'LABFORCE2', 'WKSTAT2', 'BIRTH']
36770


In [8]:
count = 0
for i in range(len(data)):
    if data[i][27]==data[i][42]:
        count += 1
print("There are", count, "observations where empstat=empstat2, so baseline accuracy for empstat2 is", count/len(data))
# (if using empstat (27) in input)

There are 30212 observations where empstat=empstat2, so baseline accuracy for empstat2 is 0.8216480826760947


In [102]:
# these are the indices of the variables the network will use

# all:
#indexNet = [5, 6, 7, 8, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]

# a few:
indexNet = [8, 15, 16]

# for empstat2:
# excluding (27, 'EMPSTAT')(28, 'LABFORCE')(29, 'OCC')(30, 'IND')(31, 'CLASSWKR')(32, 'UHRSWORKT')(33, 'WHYUNEMP')(34, 'WKSTAT')
#indexNet = [5, 6, 7, 8, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 35, 36, 37, 38]

# for marst2:
# excluding (18, 'MARST')
#indexNet = [5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]

# for birth:
# excluding none, since BIRTH is added to the data later 
#indexNet = [5, 6, 7, 8, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]

In [103]:
# generate training and testing data
# NOTE: change y targets (see comments)

dataNet = data[1:]
random.shuffle(dataNet)

Xtrain = []
ytrain = []
Xtest = []
ytest = []

for obs in range(1, 33000):
    row = []
    for i in indexNet:
        row.append(int(dataNet[obs][i]))
    Xtrain.append(row)
    #if dataNet[obs][42] in ['1', '10', '12']: # employed
    #if dataNet[obs][40] in ['1', '2', '3']: # married 
    if dataNet[obs][45] in ['1']: # born 
        ytrain.append([1])
    else:
        ytrain.append([0])
Xtrain = np.array(Xtrain)
ytrain = np.array(ytrain)

for obs in range(33001, len(dataNet)):
    row = []
    for i in indexNet:
        row.append(int(dataNet[obs][i]))
    Xtest.append(row)
    #if dataNet[obs][42] in ['1', '10', '12']: # employed 
    #if dataNet[obs][40] in ['1', '2', '3']: # married 
    if dataNet[obs][45] in ['1']: # born
        ytest.append([1])
    else:
        ytest.append([0])
Xtest = np.array(Xtest)
ytest = np.array(ytest)

In [104]:
net = Sequential()
net.add(Dense(10, input_dim=len(indexNet), activation = 'relu'))
net.add(Dense(20, activation='relu'))
net.add(Dense(30, activation='relu'))
net.add(Dense(1, activation='sigmoid'))

net.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

net.fit(Xtrain, ytrain, epochs=20, batch_size=10)

Epoch 1/20
3300/3300 [==============================] - 6s 2ms/step - loss: 0.2842 - accuracy: 0.9561
Epoch 2/20
3300/3300 [==============================] - 8s 2ms/step - loss: 0.1860 - accuracy: 0.9599
Epoch 3/20
3300/3300 [==============================] - 6s 2ms/step - loss: 0.1726 - accuracy: 0.9600
Epoch 4/20
3300/3300 [==============================] - 4s 1ms/step - loss: 0.1671 - accuracy: 0.9600
Epoch 5/20
3300/3300 [==============================] - 4s 1ms/step - loss: 0.1651 - accuracy: 0.9600
Epoch 6/20
3300/3300 [==============================] - 5s 1ms/step - loss: 0.1650 - accuracy: 0.9600
Epoch 7/20
3300/3300 [==============================] - 6s 2ms/step - loss: 0.1649 - accuracy: 0.9600
Epoch 8/20
3300/3300 [==============================] - 5s 1ms/step - loss: 0.1643 - accuracy: 0.9600
Epoch 9/20
3300/3300 [==============================] - 4s 1ms/step - loss: 0.1647 - accuracy: 0.9600
Epoch 10/20
3300/3300 [==============================] - 4s 1ms/step - loss: 0.164

In [24]:
net.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 10)                300       
_________________________________________________________________
dense_13 (Dense)             (None, 20)                220       
_________________________________________________________________
dense_14 (Dense)             (None, 30)                630       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 31        
Total params: 1,181
Trainable params: 1,181
Non-trainable params: 0
_________________________________________________________________


In [105]:
score = net.evaluate(Xtest, ytest, batch_size=100)

38/38 [==============================] - 0s 743us/step - loss: 0.1581 - accuracy: 0.9613
